In [ ]:
#importando as bibliotecas necessárias para a análise
pip install scikit-learn

In [ ]:
#importando a base de dados
import pandas as pd
tabela = pd.read_csv("clientes.csv")
display(tabela)

In [ ]:
#identificando se há valores vazios ou em formato errado
display(tabela.info())
print(tabela.columns)

In [ ]:
# a IA apenas consegue entender números e não textos. Portanto as colunas de texto precisam ser alteradas para números
# para usar uma inteligência artificial são necessários 3 passos: importar, criar e treinar
from sklearn.preprocessing import LabelEncoder

# vai transformar as colunas de texto em números
codificador = LabelEncoder()

# apenas a coluna de score_credito que não será alterada, uma vez é o foco da análise
tabela["profissao"] = codificador.fit_transform(tabela["profissao"])
tabela["mix_credito"] = codificador.fit_transform(tabela["mix_credito"])
tabela["comportamento_pagamento"] = codificador.fit_transform(tabela["comportamento_pagamento"])

# verificando se realmente todas as colunas foram modificadas
display(tabela.info())

In [ ]:
# escolha de quais colunas serão usadas para treinar o modelo, lembrando que
# y é a coluna que queremos que o modelo calcule
# x vai todas as colunas que vamos usar para prever o score de credito

#coluna a ser calculada (prevista)
y = tabela["score_credito"]

#retirada das colunas que não serão usadas na previsão e deixando apenas as que contribuem. O score_credito é a que vai ser calculada
x = tabela.drop(['score_credito', 'id_cliente'], axis=1)

from sklearn.model_selection import train_test_split

# separando os dados em treino e teste. 70% de treino para os modelos aprenderem e 30% de teste para observar se o modelo aprendeu corretamente
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y) #70% para treino e 30% para teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.3, random_state=1)

In [ ]:
# modelos selecionados para fazer o teste: Random Forest, árvore de decisão e KNN - nearest neighbors

from sklearn.ensemble import RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

modelo_forest = RandomForestClassifier() # modelo de Random Forest
modelo_knn = KNeighborsClassifier() # modelo do KNN (nearest neighbors - vizinhos mais proximos)
modelo_arvore = DecisionTreeClassifier() # modelo árvore de decisao 

# treinando os modelos
modelo_forest.fit(x_treino,y_treino) 
modelo_knn.fit(x_treino, y_treino)
modelo_arvore.fit(x_treino, y_treino)

In [ ]:
from sklearn.metrics import accuracy_score

# calculando as previsões
previsao_forest = modelo_forest.predict(x_teste)
previsao_knn = modelo_knn.predict(x_teste.to_numpy())
previsao_arvore = modelo_arvore.predict(x_teste)

# comparando as previsões com o y_teste
print(accuracy_score(y_teste, previsao_forest))
print(accuracy_score(y_teste, previsao_knn))
print(accuracy_score(y_teste, previsao_arvore))

#melhor modelo para esse projeto = Random Forest
#83% de acurácia

In [ ]:
# executando novas previsões

novos_clientes = pd.read_csv("novos_clientes.csv")
codificador = LabelEncoder()
for coluna in novos_clientes.columns:
    if novos_clientes[coluna].dtype == "object" and coluna != "score_credito":
        novos_clientes[coluna] = codificador.fit_transform(novos_clientes[coluna])
print(novos_clientes.info())

In [ ]:
#processo de previsão
previsao = modelo_forest.predict(novos_clientes)
display(novos_clientes)
print(previsao)

In [ ]:
# quais os principais fatores levados em conta para definir o score de crédito?
colunas = list(x_teste.columns)
importancia = pd.DataFrame(index=colunas, data=modelo_forest.feature_importances_)
importancia = importancia * 100
print(importancia)